# Rigidness of polyhedron

Let's check the effect on execution time of how rigid the hyperpolyhedron check is. Let's compare the results we already got (in time.ipynb) with these results, which execute the explanations with flexible polyhedron margins.

## Run these once, at the start of the notebook

In [ ]:
import logging
from time import perf_counter

import numpy as np
import pandas as pd
from rich import print

import oab

In [2]:
# make results list
results_table = []

## Personalize the following cell

In [3]:
# personalize this for running the notebook in different ways
dataset_name = "mnist"
algo_name = "RF"

how_many_images = 50  # how many images each combination of the above?

## Run these after each "personalization"
To get results on multiple datasets, algorithms, classes
### Cells *not* timed
Data loading and more

In [ ]:
(X_train, Y_train), (X_test, Y_test), (X_tree, Y_tree) = oab.get_data(dataset_name)
my_dom = oab.Domain(dataset_name, algo_name)
index = {}
points = {}
for my_class in range(10):
    index[my_class] = np.where(Y_test == my_class)[0]

### Timed cells

In [ ]:
for my_class in [int(x) for x in my_dom.classes]:
    for i, my_index in enumerate(index[my_class]):
        start = perf_counter()
        testpoint = oab.TestPoint(X_test[my_index], my_dom)
        logging.info(f"start explanation of index {my_index}")
        exp = oab.Explainer(
            testpoint, howmany=5, margins="soft"
        )  # here is the margins rigidness

        end = perf_counter()
        execution_time = end - start

        # if we failed to find a target
        if exp.target:
            crules = len(exp.target.latentdt.counterrules)
        else:
            crules = "error"

        # append into results_table
        results_table.append(
            {
                "Dataset": dataset_name,
                "Algo": algo_name,
                "Class": my_class,
                "id": my_index,
                "Time": round(execution_time, 2),
                "factuals / 5": len(exp.factuals),
                "cfact": len(exp.counterfactuals),
                "crules": crules,
            }
        )

        if i >= how_many_images - 1:
            # how many points to do per class
            break

## Run this to show the results table

In [ ]:
results_dataframe = pd.DataFrame.from_records(results_table)
results_dataframe

## Reading guide

If the **factuals generated are less than 5**, it means out of 100 generated (and discriminated) candidate factuals, less than 5 actually predicted to the same class as the point being explained. Same thing if the factuals generated are 0: out of 100 generated and discriminated images, none predicted to the correct class.

If the **counterfactuals generated are less than the counterrules**, it means one of the following:
1. one or more of those counterfactuals did not pass the discriminator or
2. one or more did not predict to a different class than the point being explained.

If **in the crules column appears "error"**, it means that the KNN was not able to find a target TreePoint in our explanation base for the failure of our 3 checks. Therefore, we fail for the entire explanation process (factuals and counterfactuals generated are zero).

If the **crules (counterrules) generated are 0**, it's an Abele failure and I can't generate any counterexemplars.